Imports and Paths

In [30]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')


# Define paths to the training, validation, and test datasets
train_path = "data/train"
val_path = "data/val"
test_path = "data/test"


Batch Size

In [14]:
# Define the batch size
batch_size = 64

Data Augmentation for Training Data

In [15]:
# Data augmentation and preprocessing for the training data
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

Load and Preprocess Training Data

In [16]:
# Load and preprocess the training data
train_batch = train_datagen.flow_from_directory(
    directory=train_path, target_size=(244, 244), batch_size=batch_size, class_mode='binary')

Found 5064 images belonging to 2 classes.


Preprocessing for Test Data

In [17]:
# Preprocessing for the test data without data augmentation
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
test_batch = test_datagen.flow_from_directory(
    directory=test_path, 
    target_size=(244, 244), 
    batch_size=batch_size, 
    class_mode='binary'
)

Found 1733 images belonging to 2 classes.


Preprocessing for Validation Data

In [18]:
# Preprocessing for the validation data without data augmentation
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
val_batch = val_datagen.flow_from_directory(
    directory=val_path, 
    target_size=(244, 244), 
    batch_size=batch_size, 
    class_mode='binary'
)

Found 755 images belonging to 2 classes.


Load Pre-trained VGG16 Model

In [19]:
# Load the VGG16 model pre-trained on ImageNet, excluding the top classification layer
vgg16_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(244, 244, 3))


Freeze VGG16 Layers

In [20]:
# Freeze the layers of the VGG16 model to prevent them from being trained
for layer in vgg16_model.layers:
    layer.trainable = False

Build Custom Model

In [21]:
# Build the custom model by adding new layers to the VGG16 base model
model = keras.Sequential([
    vgg16_model,                           # Add the pre-trained VGG16 model
    keras.layers.Flatten(),                # Flatten the output from VGG16
    keras.layers.Dense(256, activation='relu'),  # Add a fully connected layer with 256 neurons and ReLU activation
    keras.layers.Dropout(0.5),             # Add dropout for regularization to prevent overfitting
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with a single neuron for binary classification
])


Compile Model

In [22]:
# Compile the model with Adam optimizer, binary crossentropy loss, and accuracy metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Train Model

In [23]:
# Train the model with the training data and validate on the validation data
result = model.fit(
    train_batch, 
    validation_data=val_batch, 
    epochs=2, 
    verbose=1
)


Epoch 1/2
80/80 [==============================] - 1436s 18s/step - loss: 1.0635 - accuracy: 0.9619 - val_loss: 0.3768 - val_accuracy: 0.9815
Epoch 2/2
80/80 [==============================] - 1440s 18s/step - loss: 0.4265 - accuracy: 0.9765 - val_loss: 0.1381 - val_accuracy: 0.9934


Evaluate Model

In [24]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_batch, verbose=1)
print('Test accuracy:', test_acc)

28/28 [==============================] - 412s 15s/step - loss: 0.5419 - accuracy: 0.9700
Test accuracy: 0.9699942469596863


Saving Model

In [25]:
model.save("D:\project\data")

INFO:tensorflow:Assets written to: D:\project\data\assets


INFO:tensorflow:Assets written to: D:\project\data\assets


testing the model on an external photo

In [32]:
import cv2

# Load the trained model
model = keras.models.load_model('D:\project\data')

# Function to preprocess the input image
def preprocess_image(image_path):
    
    image = cv2.imread(image_path)
    image = cv2.resize(image, (244, 244))
    image = image.astype('float32')
    image /= 255.0 
    # Expand dimensions to create a batch of size 1
    image = np.expand_dims(image, axis=0)
    return image

# Path to the test image
test_image_path = 'test.JPEG'

# Preprocess the test image
test_image = preprocess_image(test_image_path)

# Make predictions
prediction = model.predict(test_image)

# Convert prediction to a readable format
predicted_class = "Without Mask" if prediction > 0.5 else "With Mask"
print("Prediction:", predicted_class)


1/1 [==============================] - 0s 423ms/step
Prediction: With Mask


In [39]:
import cv2
import numpy as np
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('D:\\project\\data')

# Define the mask detection labels
mask_dict = {0: 'Without Mask', 1: 'With Mask'}

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Preprocess the frame function
def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (244, 244))  # Resize to match model input size
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    normalized_frame = rgb_frame.astype('float32') / 255.0  # Normalize pixel values
    input_data = np.expand_dims(normalized_frame, axis=0)  # Expand dimensions to fit model input
    return input_data

# Read frames from the webcam
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    input_data = preprocess_frame(frame)

    # Perform prediction
    prediction = model.predict(input_data)

    # Convert prediction to label
    mask_label = mask_dict[int(prediction[0] > 0.5)]  # For sigmoid activation, threshold at 0.5

    # Display mask label on the frame
    cv2.putText(frame, mask_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam Mask Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 295ms/step
